In [1]:
# from datasets import load_market_data
import pandas as pd
# from modules._file_paths import OPTION_DATA
from modules._utils import load_data
from modules import HistoricalData
from modules import Portfolio
from datetime import timedelta
import numpy as np

# from vasco import plot
import glob
import os
from tqdm import tqdm

### Load Market Data

In [2]:
import timeit

load_market_data_times = []

for i in range(100):
    start_time = timeit.default_timer()
    eis_data = HistoricalData(source='eis_data', name = 'EIS DATA', underlying_instrument='BANKNIFTY',start_date='20200101', end_date='20200101')
    eis_data.load_market_data()
    elapsed = timeit.default_timer() - start_time
    
    load_market_data_times.append(elapsed)

In [3]:
np.std(load_market_data_times)

0.2035146725803996

In [4]:
np.mean(load_market_data_times)

3.4155671594695014

### Portfolio

In [5]:
import timeit

portfolio_times = []

for i in range(100):
    start_time = timeit.default_timer()
    portfolio = Portfolio(initial_cash=100000,mkt_data=eis_data)
    # call second
    portfolio.view()
    elapsed = timeit.default_timer() - start_time
    
    portfolio_times.append(elapsed)


100000 INR cash added to portfolio.

 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 *****


 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 ******************************
portfolio state
---------------
  instrument_id position current_price weighted_avg_price   value delta  \
0         00000   100000             1                  1  100000     0   

  trade_id_list  
0             0  
******************************
100000 INR cash added to portfolio.

 ******************************
portfolio 

In [6]:
np.std(portfolio_times)

0.016887163745380726

In [7]:
np.mean(portfolio_times)

0.004617690799495904

### Strategy

In [8]:
from main import get_time_interval
from datetime import datetime
from modules import Strategy

In [9]:
start_datetime = '2021-01-01 09:16:00'
end_datetime = '2021-01-01 15:30:00'

start_datetime = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')
end_datetime = datetime.strptime(end_datetime, '%Y-%m-%d %H:%M:%S')

In [10]:
time_interval_list = get_time_interval(start_datetime= start_datetime,
                                            end_datetime=end_datetime,
                                            time_delta_type="minutes",
                                            time_delta_value=1)

In [11]:
import timeit

strategy_times = []

for i in range(100):
    start_time = timeit.default_timer()
    strategy = Strategy(portfolio,
                            strategy_type='condor', 
                            param_list=[5, # otm_percentage
                                        5, # trade_interval
                                        2, # hedge_interval
                                        45, # unwind_time
                                        1, # unit_size,
                                        1], # [otm_percentage,trade_interval,hedge_interval,unwind_time,unit_size,is_mkt_maker]
                            mkt_data=eis_data,
                            underlying_instrument='BANKNIFTY',
                            time_interval_list=time_interval_list)
    elapsed = timeit.default_timer() - start_time
    
    strategy_times.append(elapsed)

In [12]:
np.std(strategy_times)

9.554571148001186e-07

In [13]:
np.mean(strategy_times)

1.2957806757185607e-06

### Blotter

In [14]:
import timeit
from modules import Blotter

blotter_times = []

for i in range(100):
    start_time = timeit.default_timer()
    blotter = Blotter(portfolio,hist_data=eis_data)
    # call third
    blotter.view()
    elapsed = timeit.default_timer() - start_time
    
    blotter_times.append(elapsed)


******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
******************************
******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
******************************
******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
******************************
******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
******************************
******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
******************************
******************************
blotter state
-------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position

In [15]:
np.std(blotter_times)

0.00025105952195154176

In [16]:
np.mean(blotter_times)

0.0008310717399581336

### Backtest

In [17]:
import timeit
from modules import Backtest

backtest_times = []

for i in range(100):
    start_time = timeit.default_timer()
    backtest = Backtest(name='condor strategy', init_from_file=False) # new change
    elapsed = timeit.default_timer() - start_time
    
    backtest_times.append(elapsed)

In [18]:
np.std(backtest_times)

7.272088064433761e-05

In [19]:
np.mean(backtest_times)

0.0005369931693712715

### Algo

In [20]:
import timeit
from modules import Algo

algo_times = []

for i in range(100):
    start_time = timeit.default_timer()
    algo = Algo(time_window=time_interval_list[:25],
            strategy=strategy,
            blotter=blotter)
    # call first
    algo.driver()
    elapsed = timeit.default_timer() - start_time
    
    algo_times.append(elapsed)

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
2021-01-01 09:16:00 # not a hedging time.

not a unwind time
2021-01-01 09:17:00 # not a trading time.
2021-01-01 09:17:00 # not a hedging time.

not a unwind time
2021-01-01 09:18:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:18:00
----------------------------------------------------------------------------------------------------
delta computed as 0
NaT
max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Colum

NaT
max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
2021-01-01 09:35:00 # not a trading time.
2021-01-01 09:35:00 # not a hedging time.

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:36:00
----------------------------------------------------------

max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
2021-01-01 09:30:00 # not a trading time.
2021-01-01 09:30:00 # not a hedging time.

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:31:00
--------------------------------------------------------------

max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
2021-01-01 09:39:00 # not a trading time.
2021-01-01 09:39:00 # not a hedging time.

not a unwind time
2021-01-01 09:40:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:40:00
----------------------------------------------------------------------------------------------------
delta computed as 0
NaT
max() arg is an empty sequence
********************************************************************************************

NaT
max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
2021-01-01 09:36:00 # not a hedging time.

not a unwind time
2021-01-01 09:37:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:37:00
----------------------------------------------------------

...

max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
2021-01-01 09:34:00 # not a trading time.
2021-01-01 09:34:00 # not a hedging time.

not a unwind time
2021-01-01 09:35:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:35:00
----------------------------------------------------------------------------------------------------
delta computed as 0
NaT
max() arg is an empty sequence
********************************************************************************************

NaT
max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
2021-01-01 09:36:00 # not a hedging time.

not a unwind time
2021-01-01 09:37:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:37:00
----------------------------------------------------------

max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
2021-01-01 09:36:00 # not a hedging time.

not a unwind time
2021-01-01 09:37:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:37:00
--------------------------------------------------------------

max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
****************************************************************************************************
generating trade strategy
----------------------------------------------------------------------------------------------------
2021-01-01 09:36:00 # not a hedging time.

not a unwind time
2021-01-01 09:37:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:37:00
--------------------------------------------------------------

NaT
max() arg is an empty sequence
****************************************************************************************************
blotter state
----------------------------------------------------------------------------------------------------
Empty DataFrame
Columns: [trade_id, time, instruement_id, position, price]
Index: []
----------------------------------------------------------------------------------------------------

not a unwind time
2021-01-01 09:38:00 # not a trading time.
2021-01-01 09:38:00 # not a hedging time.

not a unwind time
2021-01-01 09:39:00 # not a trading time.
****************************************************************************************************
initiating hedge strategy at time 2021-01-01 09:39:00
----------------------------------------------------------------------------------------------------
delta computed as 0
NaT
max() arg is an empty sequence
****************************************************************************************

In [21]:
np.std(algo_times)

0.008450792867307853

In [22]:
np.mean(algo_times)

0.10249357377091656